# Beijing Air Quality Forecasting Starter Notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Explore the training data

In this sections explore your dataset with appropiate statistics and visualisations to understand your better. Ensure that you explain output of every code cell and what it entails.

In [ ]:
print("Training Data Overview:")
train.head()

In [ ]:
train.columns

In [ ]:
# Ensure 'datetime' column is in datetime format
train['datetime'] = pd.to_datetime(train['datetime'])

test['datetime'] = pd.to_datetime(test['datetime'])

# Set the 'datetime' column as the index for better time-series handling
train.set_index('datetime', inplace=True)
# val.set_index('datetime', inplace=True)
test.set_index('datetime', inplace=True)


In [157]:
def create_temporal_features(df):
    df = df.copy()
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofweek'] = df.index.dayofweek
    df['dayofyear'] = df.index.dayofyear
    
    # Cyclical encoding for periodic features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    return df

train = create_temporal_features(train)
test = create_temporal_features(test)

In [ ]:
def handle_missing_values(df):
    df = df.copy()
    
    # For weather-related features, use interpolation
    weather_cols = ['DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
    for col in weather_cols:
        if col in df.columns:
            df[col] = df[col].interpolate(method='time')
    
    # Forward fill then backward fill for remaining
    df = df.fillna(method='ffill').fillna(method='bfill')
    
    return df

train = handle_missing_values(train)
test = handle_missing_values(test)

In [ ]:
if 'pm2.5' in train.columns:
    Q1 = train['pm2.5'].quantile(0.25)
    Q3 = train['pm2.5'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Cap outliers instead of removing them
    train['pm2.5'] = train['pm2.5'].clip(lower=max(0, lower_bound), upper=upper_bound)



In [142]:
def create_advanced_features(df, target_col='pm2.5'):
    """Create more sophisticated features"""
    df = df.copy()
    
    if target_col in df.columns:
        # Interaction features between weather variables
        df['temp_dewp_interaction'] = df['TEMP'] * df['DEWP']
        df['temp_pres_interaction'] = df['TEMP'] * df['PRES']
        df['wind_temp_interaction'] = df['Iws'] * df['TEMP']
        
        # Weather condition categories
        df['high_pressure'] = (df['PRES'] > df['PRES'].quantile(0.75)).astype(int)
        df['low_pressure'] = (df['PRES'] < df['PRES'].quantile(0.25)).astype(int)
        df['high_wind'] = (df['Iws'] > df['Iws'].quantile(0.75)).astype(int)
        df['low_wind'] = (df['Iws'] < df['Iws'].quantile(0.25)).astype(int)
        
        # Seasonal patterns
        df['winter'] = df['month'].isin([12, 1, 2]).astype(int)
        df['spring'] = df['month'].isin([3, 4, 5]).astype(int)
        df['summer'] = df['month'].isin([6, 7, 8]).astype(int)
        df['autumn'] = df['month'].isin([9, 10, 11]).astype(int)
        
        # Rush hour indicators
        df['morning_rush'] = df['hour'].isin([7, 8, 9]).astype(int)
        df['evening_rush'] = df['hour'].isin([17, 18, 19]).astype(int)
        df['weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
        
        # Advanced lag features with decay
        weights = np.exp(-np.arange(1, 13) * 0.1)  # Exponential decay
        for i, weight in enumerate(weights, 1):
            if f'{target_col}_lag{i}' in df.columns:
                df[f'{target_col}_weighted_lag{i}'] = df[f'{target_col}_lag{i}'] * weight
        
        # Trend features
        for window in [6, 12, 24]:
            if f'{target_col}_rolling_mean_{window}' in df.columns:
                df[f'{target_col}_trend_{window}'] = (
                    df[target_col] - df[f'{target_col}_rolling_mean_{window}']
                )
        
        # Fourier features for periodicity
        for period in [24, 168, 8760]:  # Daily, weekly, yearly
            df[f'sin_{period}'] = np.sin(2 * np.pi * df['hour'] / period)
            df[f'cos_{period}'] = np.cos(2 * np.pi * df['hour'] / period)
    
    return df

# train = create_advanced_features(train)



# # For test set, if it doesn't have pm2.5, use last known values from train
# if 'pm2.5' not in test.columns:
#     # Use the last few values from training data to initialize test features
#     last_train_values = train['pm2.5'].tail(24).values
#     test = create_enhanced_lag_features(test)
#     # Fill initial NaN values with appropriate values
#     for col in test.columns:
#         if 'pm2.5' in col and test[col].isna().all():
#             test[col] = np.mean(last_train_values)
# else:
#     test = create_enhanced_lag_features(test)

# # Drop rows with NaN values (mainly from lag features)
# train = train.dropna()
# test = test.fillna(method='ffill').fillna(method='bfill')

# print(f"Training data shape after feature engineering: {train.shape}")
# print(f"Test data shape after feature engineering: {test.shape}")

# train = train.dropna()
# test = test.fillna(method='ffill').fillna(method='bfill')


def add_lag_features(df, target_col="pm2.5", lags=[1, 6, 12, 24]):
    df = df.copy()
    for lag in lags:
        df[f"{target_col}_lag{lag}"] = df[target_col].shift(lag)
    return df

# 1. Add lag features first (safe)
train = add_lag_features(train, target_col="pm2.5", lags=[1,6,12,24])
test = add_lag_features(test, target_col="pm2.5", lags=[1,6,12,24])

# Drop NA rows in training
train = train.dropna()

# Fill forward/backward in test
test = test.fillna(method='ffill').fillna(method='bfill')

# 2. Now create advanced features using those lag cols
train = create_advanced_features(train, target_col="pm2.5")
test = create_advanced_features(test, target_col="pm2.5")

print(f"Training data shape: {train.shape}")
print(f"Test data shape: {test.shape}")
    


Training data shape: (30598, 91)
Test data shape: (13148, 28)


# Handle missing values


- Check the dataset for missing values and decide how to handle them.
- In this example, missing values are filled with the mean. You can experiment with other strategies.

In [143]:
# ---- Handle missing values using forward fill ----
# First, forward fill to carry previous values down
train.fillna(method='ffill', inplace=True)
test.fillna(method='ffill', inplace=True)

# Backfill as a safety net (for first few rows which may still be NaN)
train.fillna(method='bfill', inplace=True)
test.fillna(method='bfill', inplace=True)


# ---- Create lag features ----
n_lags = 3

for lag in range(1, n_lags + 1):
    train[f'pm2.5_lag{lag}'] = train['pm2.5'].shift(lag)

# If test set doesn't have pm2.5, give 0 as placeholder, else shift
for lag in range(1, n_lags + 1):
    test[f'pm2.5_lag{lag}'] = test['pm2.5'].shift(lag) if 'pm2.5' in test.columns else 0

# Drop any remaining NaNs caused by shifting
train = train.dropna()
test.fillna(0, inplace=True)  # Fill any possible NaNs in test


# Separate features and target

- Feel free to trop any non-essential columns like that you think might not contribute to modeling.

In [150]:
val_size = int(len(train) * 0.2)
train_data = train.iloc[:-val_size]
val_data = train.iloc[-val_size:]

# Separate features and target
feature_cols = [col for col in train.columns if col not in ['pm2.5', 'No']]
X_train = train_data[feature_cols].copy()
y_train = train_data['pm2.5'].copy()
X_val = val_data[feature_cols].copy()
y_val = val_data['pm2.5'].copy()

for col in X_train.columns:
    # Replace infinities with NaN first
    X_train[col] = X_train[col].replace([np.inf, -np.inf], np.nan)
    X_val[col] = X_val[col].replace([np.inf, -np.inf], np.nan)
    
    # For numeric columns, use quantile-based clipping
    if X_train[col].dtype in ['float64', 'float32', 'int64', 'int32']:
        q01 = X_train[col].quantile(0.01)
        q99 = X_train[col].quantile(0.99)
        if not pd.isna(q01) and not pd.isna(q99) and q01 != q99:
            X_train[col] = X_train[col].clip(lower=q01, upper=q99)
            X_val[col] = X_val[col].clip(lower=q01, upper=q99)

# Fill remaining NaN values with column means
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_train.mean())  # Use training means for validation

# Final check for problematic values
print(f"After cleaning:")
print(f"X_train infinite values: {np.isinf(X_train.values).sum()}")
print(f"X_val infinite values: {np.isinf(X_val.values).sum()}")
print(f"X_train NaN values: {X_train.isnull().sum().sum()}")
print(f"X_val NaN values: {X_val.isnull().sum().sum()}")

# Feature selection (with error handling)
if len(feature_cols) > 50:
    try:
        print("Attempting feature selection...")
        selector = SelectKBest(score_func=f_regression, k=min(50, len(feature_cols)))
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_val_selected = selector.transform(X_val)
        selected_features = X_train.columns[selector.get_support()]
        print(f" Selected {len(selected_features)} features out of {len(feature_cols)}")
        X_train = pd.DataFrame(X_train_selected, columns=selected_features, index=X_train.index)
        X_val = pd.DataFrame(X_val_selected, columns=selected_features, index=X_val.index)
    except Exception as e:
        print(f" Feature selection failed: {e}")
        print("Using all features without selection...")
        
print(f"Final feature set size: {X_train.shape[1]}")
print(f"Features: {list(X_train.columns)}")

# Now continue with the rest of your notebook...
# The scaling and sequence creation should work now

# Use this instead of your current scaling section:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Target scaling
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_val_scaled = target_scaler.transform(y_val.values.reshape(-1, 1)).flatten()

def create_sequences(X, y, sequence_length=24):
    X_seq, y_seq = [], []
    for i in range(sequence_length, len(X)):
        X_seq.append(X[i-sequence_length:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

sequence_length = 24
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)

print(f"\n Final data shapes:")
print(f"X_train_seq: {X_train_seq.shape}")
print(f"y_train_seq: {y_train_seq.shape}")
print(f"X_val_seq: {X_val_seq.shape}")
print(f"y_val_seq: {y_val_seq.shape}")

After cleaning:
X_train infinite values: 0
X_val infinite values: 0
X_train NaN values: 0
X_val NaN values: 0
Attempting feature selection...
 Selected 50 features out of 90
Final feature set size: 50
Features: ['DEWP', 'Iws', 'cbwd_NW', 'pm2.5_lag1', 'pm2.5_lag2', 'pm2.5_lag3', 'pm2.5_lag4', 'pm2.5_lag5', 'pm2.5_lag6', 'pm2.5_lag7', 'pm2.5_lag8', 'pm2.5_lag9', 'pm2.5_lag10', 'pm2.5_lag11', 'pm2.5_lag12', 'pm2.5_rolling_mean_3', 'pm2.5_rolling_std_3', 'pm2.5_rolling_max_3', 'pm2.5_rolling_min_3', 'pm2.5_rolling_mean_6', 'pm2.5_rolling_std_6', 'pm2.5_rolling_max_6', 'pm2.5_rolling_min_6', 'pm2.5_rolling_mean_12', 'pm2.5_rolling_max_12', 'pm2.5_rolling_min_12', 'pm2.5_rolling_mean_24', 'pm2.5_rolling_max_24', 'pm2.5_rolling_min_24', 'pm2.5_ewm_3', 'pm2.5_ewm_12', 'pm2.5_pct_change_24', 'high_wind', 'pm2.5_weighted_lag1', 'pm2.5_weighted_lag2', 'pm2.5_weighted_lag3', 'pm2.5_weighted_lag4', 'pm2.5_weighted_lag5', 'pm2.5_weighted_lag6', 'pm2.5_weighted_lag7', 'pm2.5_weighted_lag8', 'pm2.5_w

In [149]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Target scaling (often helps with LSTM)
target_scaler = StandardScaler()
y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_val_scaled = target_scaler.transform(y_val.values.reshape(-1, 1)).flatten()

def create_sequences(X, y, sequence_length=24):
    X_seq, y_seq = [], []
    for i in range(sequence_length, len(X)):
        X_seq.append(X[i-sequence_length:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

sequence_length = 24  # Use 24 hours of history
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, sequence_length)

print(f"Training sequences shape: {X_train_seq.shape}")
print(f"Validation sequences shape: {X_val_seq.shape}")

SyntaxError: unterminated string literal (detected at line 3) (3583020122.py, line 3)

# Build model

Below is a simple LSTM model. Your task is to experiment with different parameters like, numbers of layers, units, activation functions, and optimizers, etc to get the best performing model. Experiment with other optimizers (e.g., SGD) or hyperparameters to improve performance.

In [158]:
# def create_improved_model(input_shape):
#     model = Sequential([
#         LSTM(128, return_sequences=True, input_shape=input_shape),
#         Dropout(0.2),
#         BatchNormalization(),
        
#         LSTM(64, return_sequences=True),
#         Dropout(0.2),
#         BatchNormalization(),
        
#         LSTM(32),
#         Dropout(0.2),
#         BatchNormalization(),
        
#         Dense(32, activation='relu'),
#         Dropout(0.2),
#         Dense(16, activation='relu'),
#         Dense(1)
#     ])
    
#     return model

# model = create_improved_model((sequence_length, X_train_seq.shape[2]))

# # Custom optimizer with learning rate scheduling
# optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# model.compile(
#     optimizer=optimizer,
#     loss='huber',  # Huber loss is more robust to outliers than MSE
#     metrics=['mae']
# )

# model.summary()



class AdvancedEnsemble:
    def __init__(self):
        self.models = {}
        self.weights = {}
        self.scalers = {}
    
    def create_lstm_model(self, input_shape, model_type='standard'):
        """Create different LSTM architectures"""
        if model_type == 'standard':
            model = Sequential([
                LSTM(128, return_sequences=True, input_shape=input_shape),
                Dropout(0.3),
                LSTM(64, return_sequences=True),
                Dropout(0.3),
                LSTM(32),
                Dense(50, activation='relu'),
                Dropout(0.2),
                Dense(1)
            ])
        
        elif model_type == 'bidirectional':
            model = Sequential([
                Bidirectional(LSTM(64, return_sequences=True), input_shape=input_shape),
                Dropout(0.3),
                Bidirectional(LSTM(32)),
                Dense(50, activation='relu'),
                Dropout(0.2),
                Dense(1)
            ])
        
        elif model_type == 'cnn_lstm':
            model = Sequential([
                Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
                MaxPooling1D(pool_size=2),
                LSTM(64, return_sequences=True),
                Dropout(0.3),
                LSTM(32),
                Dense(50, activation='relu'),
                Dense(1)
            ])
        
        return model
    
    def create_tree_models(self):
        """Create tree-based models for ensemble"""
        return {
            'xgb': xgb.XGBRegressor(
                n_estimators=1000,
                max_depth=6,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42
            ),
            'lgb': lgb.LGBMRegressor(
                n_estimators=1000,
                max_depth=6,
                learning_rate=0.1,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                verbose=-1
            ),
            'rf': RandomForestRegressor(
                n_estimators=500,
                max_depth=15,
                random_state=42,
                n_jobs=-1
            )
        }
    
    def train_ensemble(self, X_train_seq, y_train_seq, X_val_seq, y_val_seq, 
                      X_train_flat, y_train_flat, X_val_flat, y_val_flat):
        """Train multiple models for ensemble"""
        
        # Train LSTM models
        input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
        
        for model_type in ['standard', 'bidirectional', 'cnn_lstm']:
            print(f"Training LSTM {model_type}...")
            model = self.create_lstm_model(input_shape, model_type)
            model.compile(optimizer=Adam(0.001), loss='huber', metrics=['mae'])
            
            early_stop = EarlyStopping(patience=15, restore_best_weights=True)
            model.fit(
                X_train_seq, y_train_seq,
                validation_data=(X_val_seq, y_val_seq),
                epochs=100, batch_size=64,
                callbacks=[early_stop], verbose=0
            )
            
            self.models[f'lstm_{model_type}'] = model
        
        # Train tree-based models
        tree_models = self.create_tree_models()
        
        for name, model in tree_models.items():
            print(f"Training {name}...")
            model.fit(X_train_flat, y_train_flat)
            self.models[name] = model
        
        # Calculate ensemble weights based on validation performance
        self.calculate_weights(X_val_seq, X_val_flat, y_val_flat)
    
    def calculate_weights(self, X_val_seq, X_val_flat, y_val):
        """Calculate optimal ensemble weights"""
        predictions = {}
        
        # Get predictions from each model
        for name, model in self.models.items():
            if 'lstm' in name:
                pred = model.predict(X_val_seq, verbose=0).flatten()
            else:
                pred = model.predict(X_val_flat)
            
            predictions[name] = pred
            rmse = np.sqrt(mean_squared_error(y_val, pred))
            print(f"{name} RMSE: {rmse:.4f}")
        
        # Simple inverse error weighting
        errors = {}
        for name, pred in predictions.items():
            errors[name] = np.sqrt(mean_squared_error(y_val, pred))
        
        total_inv_error = sum(1/error for error in errors.values())
        self.weights = {name: (1/error)/total_inv_error for name, error in errors.items()}
        
        print("Ensemble weights:", self.weights)
    
    def predict(self, X_seq, X_flat):
        """Make ensemble predictions"""
        predictions = []
        
        for name, model in self.models.items():
            if 'lstm' in name:
                pred = model.predict(X_seq, verbose=0).flatten()
            else:
                pred = model.predict(X_flat)
            
            predictions.append(pred * self.weights[name])
        
        return np.sum(predictions, axis=0)

# ================================
# 3. ADVANCED PREPROCESSING
# ================================

def create_multiple_scales(X_train, X_val, X_test):
    """Create data at multiple time scales"""
    scalers = {}
    scaled_data = {}
    
    # Standard scaling
    scaler_std = StandardScaler()
    scaled_data['standard'] = {
        'train': scaler_std.fit_transform(X_train),
        'val': scaler_std.transform(X_val),
        'test': scaler_std.transform(X_test)
    }
    scalers['standard'] = scaler_std
    
    # Robust scaling
    scaler_robust = RobustScaler()
    scaled_data['robust'] = {
        'train': scaler_robust.fit_transform(X_train),
        'val': scaler_robust.transform(X_val),
        'test': scaler_robust.transform(X_test)
    }
    scalers['robust'] = scaler_robust
    
    return scaled_data, scalers

def create_multi_scale_sequences(data, sequence_lengths=[12, 24, 48]):
    """Create sequences of different lengths"""
    sequences = {}
    
    for seq_len in sequence_lengths:
        X_seq, y_seq = [], []
        for i in range(seq_len, len(data['X'])):
            X_seq.append(data['X'][i-seq_len:i])
            y_seq.append(data['y'][i])
        
        sequences[seq_len] = {
            'X': np.array(X_seq),
            'y': np.array(y_seq)
        }
    
    return sequences

# ================================
# 4. ADVANCED VALIDATION STRATEGY
# ================================

def time_series_cv(train_data, n_splits=5):
    """Time series cross-validation"""
    total_size = len(train_data)
    test_size = total_size // (n_splits + 1)
    
    splits = []
    for i in range(n_splits):
        train_end = total_size - (n_splits - i) * test_size
        test_start = train_end
        test_end = test_start + test_size
        
        train_idx = list(range(0, train_end))
        test_idx = list(range(test_start, test_end))
        
        splits.append((train_idx, test_idx))
    
    return splits

# ================================
# 5. HYPERPARAMETER OPTIMIZATION
# ================================

def optimize_hyperparameters(X_train_seq, y_train_seq, X_val_seq, y_val_seq):
    """Simple grid search for key hyperparameters"""
    
    best_score = float('inf')
    best_params = {}
    
    # Define parameter grid
    param_grid = {
        'lstm_units': [64, 128, 256],
        'dropout_rate': [0.2, 0.3, 0.4],
        'learning_rate': [0.001, 0.01, 0.1],
        'batch_size': [32, 64, 128]
    }
    
    # Simple grid search (you might want to use more sophisticated methods)
    for lstm_units in param_grid['lstm_units']:
        for dropout_rate in param_grid['dropout_rate']:
            for lr in param_grid['learning_rate']:
                for batch_size in param_grid['batch_size']:
                    
                    model = Sequential([
                        LSTM(lstm_units, return_sequences=True, 
                             input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
                        Dropout(dropout_rate),
                        LSTM(lstm_units//2),
                        Dense(1)
                    ])
                    
                    model.compile(optimizer=Adam(lr), loss='mse')
                    
                    history = model.fit(
                        X_train_seq, y_train_seq,
                        validation_data=(X_val_seq, y_val_seq),
                        epochs=10, batch_size=batch_size,
                        verbose=0
                    )
                    
                    val_loss = min(history.history['val_loss'])
                    
                    if val_loss < best_score:
                        best_score = val_loss
                        best_params = {
                            'lstm_units': lstm_units,
                            'dropout_rate': dropout_rate,
                            'learning_rate': lr,
                            'batch_size': batch_size
                        }
    
    print("Best parameters:", best_params)
    return best_params

# ================================
# 6. POST-PROCESSING TECHNIQUES
# ================================

def post_process_predictions(predictions, y_train):
    """Apply post-processing to improve predictions"""
    
    # Clip to reasonable range based on training data
    train_min, train_max = y_train.min(), y_train.max()
    predictions = np.clip(predictions, train_min * 0.5, train_max * 1.5)
    
    # Apply smoothing to reduce noise
    from scipy.ndimage import gaussian_filter1d
    predictions_smooth = gaussian_filter1d(predictions, sigma=1.0)
    
    # Blend original and smoothed predictions
    predictions_final = 0.7 * predictions + 0.3 * predictions_smooth
    
    return predictions_final

# ================================
# 7. MAIN TRAINING PIPELINE
# ================================

def train_advanced_model(train, test, target_scaler):
    """Complete advanced training pipeline"""
    
    # 1. Advanced feature engineering
    print("Creating advanced features...")
    train_advanced = create_advanced_features(train)
    test_advanced = create_advanced_features(test)
    
    # 2. Data preparation
    val_size = int(len(train_advanced) * 0.2)
    train_data = train_advanced.iloc[:-val_size]
    val_data = train_advanced.iloc[-val_size:]
    
    feature_cols = [col for col in train_advanced.columns if col not in ['pm2.5', 'No']]
    
    X_train = train_data[feature_cols]
    y_train = train_data['pm2.5']
    X_val = val_data[feature_cols]
    y_val = val_data['pm2.5']
    
    # Clean data
    for col in X_train.columns:
        X_train[col] = X_train[col].replace([np.inf, -np.inf], np.nan)
        X_val[col] = X_val[col].replace([np.inf, -np.inf], np.nan)
    
    X_train = X_train.fillna(X_train.mean())
    X_val = X_val.fillna(X_train.mean())
    
    # 3. Multiple scaling strategies
    X_test = test_advanced[feature_cols]
    X_test = X_test.fillna(X_train.mean())
    
    scaled_data, scalers = create_multiple_scales(X_train, X_val, X_test)
    
    # Use robust scaling for main model
    X_train_scaled = scaled_data['robust']['train']
    X_val_scaled = scaled_data['robust']['val']
    X_test_scaled = scaled_data['robust']['test']
    
    # Scale targets
    y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_val_scaled = target_scaler.transform(y_val.values.reshape(-1, 1)).flatten()
    
    # 4. Create sequences
    def create_sequences(X, y, seq_len=24):
        X_seq, y_seq = [], []
        for i in range(seq_len, len(X)):
            X_seq.append(X[i-seq_len:i])
            y_seq.append(y[i])
        return np.array(X_seq), np.array(y_seq)
    
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled)
    X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled)
    
    # Also prepare flat data for tree models
    X_train_flat = X_train_scaled[24:]  # Skip first 24 to match sequence data
    X_val_flat = X_val_scaled[24:]
    y_train_flat = y_train_scaled[24:]
    y_val_flat = y_val_scaled[24:]
    
    # 5. Train ensemble
    ensemble = AdvancedEnsemble()
    ensemble.train_ensemble(
        X_train_seq, y_train_seq, X_val_seq, y_val_seq,
        X_train_flat, y_train_flat, X_val_flat, y_val_flat
    )
    
    # 6. Make test predictions
    def create_test_sequences(X_test_scaled, X_train_scaled, seq_len=24):
        combined = np.vstack([X_train_scaled[-seq_len:], X_test_scaled])
        sequences = []
        for i in range(seq_len, len(combined)):
            sequences.append(combined[i-seq_len:i])
        return np.array(sequences)
    
    X_test_seq = create_test_sequences(X_test_scaled, X_train_scaled)
    X_test_flat = X_test_scaled
    
    test_pred_scaled = ensemble.predict(X_test_seq, X_test_flat)
    test_pred = target_scaler.inverse_transform(test_pred_scaled.reshape(-1, 1)).flatten()
    
    # 7. Post-processing
    test_pred = post_process_predictions(test_pred, y_train.values)
    test_pred = np.clip(test_pred, 0, 500).astype(int)
    
    return test_pred, ensemble


In [ ]:


early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

lr_reducer = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=8,
    min_lr=1e-7,
    verbose=1
)

# Train the model
history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping, lr_reducer],
    verbose=1
)

# Make predictions
train_pred_scaled = model.predict(X_train_seq)
val_pred_scaled = model.predict(X_val_seq)

# Inverse transform predictions
train_pred = target_scaler.inverse_transform(train_pred_scaled).flatten()
val_pred = target_scaler.inverse_transform(val_pred_scaled).flatten()

# Get actual values (accounting for sequence offset)
y_train_actual = y_train.iloc[sequence_length:].values
y_val_actual = y_val.iloc[sequence_length:].values

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val_actual, val_pred))
val_mae = mean_absolute_error(y_val_actual, val_pred)

# Persistence baseline
persistence_pred = y_val.shift(1).iloc[sequence_length:].values
persistence_rmse = np.sqrt(mean_squared_error(y_val_actual, persistence_pred))

# Plotting
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Training history
axes[0, 0].plot(history.history['loss'], label='Training Loss')
axes[0, 0].plot(history.history['val_loss'], label='Validation Loss')
axes[0, 0].set_title('Training History')
axes[0, 0].set_xlabel('Epochs')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()

# Validation predictions vs actual
axes[0, 1].scatter(y_val_actual, val_pred, alpha=0.5)
axes[0, 1].plot([y_val_actual.min(), y_val_actual.max()], 
                [y_val_actual.min(), y_val_actual.max()], 'r--', lw=2)
axes[0, 1].set_xlabel('Actual PM2.5')
axes[0, 1].set_ylabel('Predicted PM2.5')
axes[0, 1].set_title('Validation: Predicted vs Actual')

# Time series plot
axes[1, 0].plot(y_val_actual[:200], label='Actual', alpha=0.7)
axes[1, 0].plot(val_pred[:200], label='Predicted', alpha=0.7)
axes[1, 0].set_title('Time Series Comparison (First 200 points)')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('PM2.5')
axes[1, 0].legend()

# Residuals
residuals = y_val_actual - val_pred
axes[1, 1].hist(residuals, bins=30, alpha=0.7)
axes[1, 1].set_title('Residuals Distribution')
axes[1, 1].set_xlabel('Residual')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"Training RMSE: {train_rmse:.2f}")
print(f"Validation RMSE: {val_rmse:.2f}")
print(f"Validation MAE: {val_mae:.2f}")
print(f"Persistence Baseline RMSE: {persistence_rmse:.2f}")
print(f"Improvement over baseline: {((persistence_rmse - val_rmse) / persistence_rmse * 100):.1f}%")


Epoch 1/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 16s 42ms/step - loss: 0.0368 - mae: 0.1819 - val_loss: 0.0573 - val_mae: 0.2527 - learning_rate: 5.0000e-04
Epoch 2/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - loss: 0.0355 - mae: 0.1787 - val_loss: 0.0682 - val_mae: 0.2726 - learning_rate: 5.0000e-04
Epoch 3/100
383/383 ━━━━━━━━━━━━━━━━━━━━ 16s 42ms/step - loss: 0.0355 - mae: 0.1772 - val_loss: 0.0634 - val_mae: 0.2681 - learning_rate: 5.0000e-04
Epoch 4/100
200/383 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss: 0.0332 - mae: 0.1757

In [153]:
def create_sliding_window_test(test_df, lookback):
    X_test = []
    # Use the same feature selection as training (drop 'No' and 'pm2.5' if present)
    features_to_drop = ['No']
    if 'pm2.5' in test_df.columns:
        features_to_drop.append('pm2.5')
    test_values = test_df.drop(features_to_drop, axis=1).values
    
    for i in range(lookback, len(test_df)):
        X_test.append(test_values[i-lookback:i])
    return np.array(X_test)


In [154]:
def create_full_predictions(test_df, model, lookback):
    # Drop non-feature columns
    test_features = test_df.drop(['No'], axis=1, errors='ignore')
    if 'pm2.5' in test_features.columns:
        test_features = test_features.drop(['pm2.5'], axis=1)

    values = test_features.values
    n = len(values)
    preds = []

    # Pad the first `lookback` rows with the first valid window prediction
    first_window = values[:lookback].reshape(1, lookback, -1)
    first_pred = model.predict(first_window, verbose=0)[0, 0]
    preds.extend([first_pred] * lookback)

    # Build sliding windows for the rest (batch all at once)
    windows = np.array([
        values[i - lookback:i] for i in range(lookback, n)
    ])
    batch_preds = model.predict(windows, verbose=0).flatten()
    preds.extend(batch_preds.tolist())

    return np.array(preds)


In [156]:
def create_test_sequences_simple(test_df, X_train_scaled, scaler, sequence_length, selected_features):
    
    test_prep = test_df.copy()
    
    # Add missing features that were selected during training
    missing_features = [col for col in selected_features if col not in test_prep.columns]
    
    if missing_features:
        print(f"Adding {len(missing_features)} missing features to test data")
        
        # Use simple defaults for missing features
        for col in missing_features:
            if 'pm2.5' in col:
                if 'lag' in col:
                    test_prep[col] = 50  # Reasonable PM2.5 default
                elif 'rolling' in col:
                    test_prep[col] = 50
                elif 'ewm' in col:
                    test_prep[col] = 50
                elif 'pct_change' in col:
                    test_prep[col] = 0
                else:
                    test_prep[col] = 0
            else:
                test_prep[col] = 0
    
    # Select only the features that were used in training
    X_test = test_prep[selected_features]
    
    # Clean test data
    for col in X_test.columns:
        X_test[col] = X_test[col].replace([np.inf, -np.inf], np.nan)
        X_test[col] = X_test[col].fillna(X_test[col].mean() if X_test[col].mean() == X_test[col].mean() else 0)
    
    print(f"Test data shape before scaling: {X_test.shape}")
    
    # Scale test features
    X_test_scaled = scaler.transform(X_test)
    
    # Use the last sequence_length rows from training data
    train_tail = X_train_scaled[-sequence_length:]
    
    # Combine training tail with test data
    combined_data = np.vstack([train_tail, X_test_scaled])
    
    # Create sequences
    sequences = []
    for i in range(sequence_length, len(combined_data)):
        sequences.append(combined_data[i-sequence_length:i])
    
    return np.array(sequences)

# Use the simplified function
print("Creating test sequences...")
X_test_seq = create_test_sequences_simple(
    test, 
    X_train_scaled,  # Use the already scaled training data
    scaler, 
    sequence_length, 
    selected_features=X_train.columns.tolist()  # Features that were actually used
)

print(f"Test sequences shape: {X_test_seq.shape}")

# Make predictions
test_pred_scaled = model.predict(X_test_seq, verbose=1)
test_pred = target_scaler.inverse_transform(test_pred_scaled).flatten()

# Ensure reasonable predictions
test_pred = np.clip(test_pred, 0, 500)
test_pred = np.round(test_pred).astype(int)

print(f"Test predictions - Min: {test_pred.min()}, Max: {test_pred.max()}, Mean: {test_pred.mean():.1f}")

# Create submission
submission = pd.DataFrame({
    'row ID': test.index.strftime('%Y-%m-%d %-H:%M:%S'),
    'pm2.5': test_pred
})

print(f"Submission shape: {submission.shape}")
submission.to_csv('data/final_submission.csv', index=False)

print(submission.head())

Creating test sequences...
Adding 41 missing features to test data
Test data shape before scaling: (13148, 50)
Test sequences shape: (13148, 24, 50)
411/411 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
Test predictions - Min: 26, Max: 214, Mean: 33.4
Submission shape: (13148, 2)
               row ID  pm2.5
0  2013-07-02 4:00:00    214
1  2013-07-02 5:00:00     72
2  2013-07-02 6:00:00     26
3  2013-07-02 7:00:00     27
4  2013-07-02 8:00:00     29
